In [1]:
import tensorflow as tf
import pickle
import sys
import matplotlib.pyplot as plt1 
import numpy as np
from IPython.display import Audio, display
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate, Conv1D,Conv2D, Flatten, Reshape, Embedding, GRU, SpatialDropout1D, LSTM, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from itertools import permutations
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.stats import trim_mean

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
full = pd.read_csv("../data/drug_class_identification/all3.csv")
full = full.dropna()
full['atc'] = full['atc'].apply(lambda x : x[0])
full.head()

,smiles,name,id,inchi_key,atc
75,NC12CC3CC(CC(C3)C1)C2,amantadine,BRD-K70330367,DKNWSYNQZKUICI-UHFFFAOYSA-N,N
291,CS(=O)(=O)OCCCCOS(C)(=O)=O,busulfan,BRD-K23204545,COVZYZSDYWQREU-UHFFFAOYSA-N,L
322,CCCC(C)(COC(N)=O)COC(=O)NC(C)C,carisoprodol,BRD-A99939097,OFZCIYFFPZCNJE-UHFFFAOYSA-N,M
491,CCN(CC)C(=O)N1CCN(C)CC1,diethylcarbamazine,BRD-K45542189,RCKMWOKWVGPNJF-UHFFFAOYSA-N,P
510,CCN(CC)C(=S)SSC(=S)N(CC)CC,disulfiram,BRD-K32744045,AUZONCFQVSMFAP-UHFFFAOYSA-N,P


In [3]:
# Define our own plot function
def scatter(x, y, subtitle=None):
    le = LabelEncoder()
    labels = le.fit_transform(y)

    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 3))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(3):
        # Position of each label.
        xtext, ytext = trim_mean(x[labels == i, :], axis=0, proportiontocut=0.2)
        letter = le.inverse_transform([i])[0]
        txt = ax.text(xtext, ytext, str(letter), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [4]:
snn = pd.read_csv('EMB_triplet_3_500_16_0.9_5-50')
snn['id'] = snn['pert_id']
snn1 = snn.groupby(['id']).mean()
snn1.shape

(2170, 17)

In [5]:
snn1.head()

,Unnamed: 0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16
id,,,,,,,,,,,,,,,,,
BRD-A00147595,20.5,-0.051912,-0.046225,-0.079341,0.010777,0.199179,0.063099,-0.008241,-0.000671,0.207154,-0.132532,-0.337864,-0.068388,0.055600,0.015693,0.483786,0.033555
BRD-A00218260,62.5,-0.007211,-0.019740,-0.087918,0.043078,0.186389,0.050391,-0.042841,-0.038788,0.219889,-0.129041,-0.340753,-0.050279,0.034886,0.055037,0.480075,0.092194
BRD-A00376169,104.5,-0.035873,-0.038781,-0.071005,0.011709,0.188347,0.062192,-0.015706,-0.001051,0.206982,-0.126873,-0.332761,-0.050283,0.053545,0.025172,0.478171,0.048873
BRD-A00546892,146.5,-0.083619,-0.048509,-0.197024,0.099494,0.271578,0.030857,-0.031520,-0.126297,0.232526,-0.192449,-0.365771,-0.211621,0.006241,0.019896,0.507215,0.036830
BRD-A00938334,188.5,-0.008202,-0.008061,-0.162835,0.113867,0.216934,0.018615,-0.071992,-0.136495,0.225272,-0.158428,-0.337248,-0.135821,-0.009357,0.072566,0.464084,0.114936


In [6]:
result_snn = pd.merge(snn, full, how="inner").drop(['id','smiles','name','inchi_key','pert_id','Unnamed: 0'],axis=1).dropna()
result_snn = result_snn[result_snn.atc.isin(['C','L','N'])]

In [7]:
y_snn = result_snn['atc']
X_snn = result_snn.drop(['atc'],axis=1)
X_snn = np.asarray(X_snn)
result_snn.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,atc
84,-0.183448,-0.088374,-0.298852,0.141471,0.356243,0.013249,-0.001803,-0.207486,0.229378,-0.251088,-0.377218,-0.378206,-0.015539,-0.035752,0.523691,-0.048028,N
85,-0.185150,-0.092180,-0.293811,0.133686,0.357638,0.017342,0.002558,-0.192717,0.231722,-0.250480,-0.383006,-0.374091,-0.008439,-0.038505,0.529485,-0.051879,N
86,0.255648,0.131313,-0.080232,0.183721,0.105543,0.000734,-0.235274,-0.185011,0.297990,-0.094100,-0.384150,0.106757,-0.053028,0.279614,0.506027,0.427559,N
87,-0.130725,-0.069101,-0.291202,0.150245,0.345000,0.016205,-0.031880,-0.201702,0.259922,-0.250814,-0.409143,-0.334909,-0.017611,0.002179,0.547097,0.010761,N
88,0.240981,0.119331,-0.100761,0.189807,0.118800,-0.001084,-0.229800,-0.190443,0.297566,-0.106891,-0.392855,0.074723,-0.052318,0.271000,0.520312,0.414156,N


In [8]:
X_snn.shape
le = LabelEncoder()
labels = le.fit_transform(y_snn)

In [9]:
x_train_snn, x_test_snn, y_train_snn, y_test_snn = train_test_split(X_snn,labels)

x_train_flat_snn = x_train_snn.reshape(-1,16)
x_test_flat_snn = x_test_snn.reshape(-1,16)

tsne = TSNE()
train_tsne_embeds_snn = tsne.fit_transform(x_train_flat_snn[:1000])
scatter(train_tsne_embeds_snn, y_train_snn[:1000], "Samples from Training Data LINCS (Triplet) Before")

eval_tsne_embeds_snn = tsne.fit_transform(x_test_flat_snn[:1000])
scatter(eval_tsne_embeds_snn, y_test_snn[:1000], "Samples from Validation Data LINCS (Triplet) Before")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [10]:
def baseNetwork(op):    
        model = Sequential()
        model.add(Dense(50, activation='softmax', input_dim=16))
        model.add(Dropout(0.5))
        model.add(Dense(20, activation='softmax'))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))
        model.add(Dropout(0.5))
        model.add(Dense(op, activation='softmax'))
        return model

In [11]:
model = baseNetwork(3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                850       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total para

In [12]:
model.fit(x_train_snn,y_train_snn, epochs=500)

Epoch 1/500
18819/18819 [==============================] - 2s 88us/step - loss: 1.0483 - acc: 0.4922
Epoch 2/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0353 - acc: 0.5121
Epoch 3/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0325 - acc: 0.5121
Epoch 4/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0301 - acc: 0.5121
Epoch 5/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0261 - acc: 0.5121
Epoch 6/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0208 - acc: 0.5121
Epoch 7/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0197 - acc: 0.5121
Epoch 8/500
18819/18819 [==============================] - 1s 75us/step - loss: 1.0165 - acc: 0.5120
Epoch 9/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0142 - acc: 0.5122
Epoch 10/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0116 - a

18819/18819 [==============================] - 1s 59us/step - loss: 1.0011 - acc: 0.5121
Epoch 82/500
18819/18819 [==============================] - 1s 58us/step - loss: 1.0027 - acc: 0.5121
Epoch 83/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0018 - acc: 0.5121
Epoch 84/500
18819/18819 [==============================] - 1s 58us/step - loss: 1.0005 - acc: 0.5121
Epoch 85/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0026 - acc: 0.5121
Epoch 86/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0000 - acc: 0.5121
Epoch 87/500
18819/18819 [==============================] - 1s 58us/step - loss: 1.0009 - acc: 0.5121
Epoch 88/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0012 - acc: 0.5121
Epoch 89/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0018 - acc: 0.5121
Epoch 90/500
18819/18819 [==============================] - 1s 67us/step - loss: 1.0011 - acc: 

18819/18819 [==============================] - 1s 58us/step - loss: 1.0002 - acc: 0.5121
Epoch 161/500
18819/18819 [==============================] - 1s 58us/step - loss: 0.9995 - acc: 0.5121
Epoch 162/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0007 - acc: 0.5119
Epoch 163/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9999 - acc: 0.5120
Epoch 164/500
18819/18819 [==============================] - 1s 60us/step - loss: 1.0013 - acc: 0.5120
Epoch 165/500
18819/18819 [==============================] - 1s 58us/step - loss: 1.0001 - acc: 0.5121
Epoch 166/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0017 - acc: 0.5121
Epoch 167/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0003 - acc: 0.5121
Epoch 168/500
18819/18819 [==============================] - 1s 59us/step - loss: 1.0010 - acc: 0.5121
Epoch 169/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.001

18819/18819 [==============================] - 1s 62us/step - loss: 1.0018 - acc: 0.5121
Epoch 240/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9995 - acc: 0.5120
Epoch 241/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.9983 - acc: 0.5120
Epoch 242/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9994 - acc: 0.5118
Epoch 243/500
18819/18819 [==============================] - 1s 65us/step - loss: 1.0010 - acc: 0.5120
Epoch 244/500
18819/18819 [==============================] - 1s 65us/step - loss: 0.9997 - acc: 0.5120
Epoch 245/500
18819/18819 [==============================] - 1s 66us/step - loss: 1.0001 - acc: 0.5120
Epoch 246/500
18819/18819 [==============================] - 1s 64us/step - loss: 1.0010 - acc: 0.5121
Epoch 247/500
18819/18819 [==============================] - 1s 64us/step - loss: 0.9997 - acc: 0.5121
Epoch 248/500
18819/18819 [==============================] - 1s 64us/step - loss: 1.000

18819/18819 [==============================] - 1s 59us/step - loss: 0.9988 - acc: 0.5121
Epoch 319/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9982 - acc: 0.5121
Epoch 320/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9983 - acc: 0.5121
Epoch 321/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9989 - acc: 0.5121
Epoch 322/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9986 - acc: 0.5118
Epoch 323/500
18819/18819 [==============================] - 1s 64us/step - loss: 0.9979 - acc: 0.5120
Epoch 324/500
18819/18819 [==============================] - 1s 70us/step - loss: 0.9982 - acc: 0.5119
Epoch 325/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9976 - acc: 0.5121
Epoch 326/500
18819/18819 [==============================] - 1s 61us/step - loss: 1.0006 - acc: 0.5120
Epoch 327/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.999

18819/18819 [==============================] - 1s 73us/step - loss: 0.9986 - acc: 0.5121
Epoch 398/500
18819/18819 [==============================] - 1s 70us/step - loss: 0.9991 - acc: 0.5120
Epoch 399/500
18819/18819 [==============================] - 1s 65us/step - loss: 0.9983 - acc: 0.5121
Epoch 400/500
18819/18819 [==============================] - 1s 64us/step - loss: 0.9989 - acc: 0.5121
Epoch 401/500
18819/18819 [==============================] - 1s 65us/step - loss: 0.9980 - acc: 0.5121
Epoch 402/500
18819/18819 [==============================] - 1s 62us/step - loss: 0.9983 - acc: 0.5120
Epoch 403/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.9978 - acc: 0.5121
Epoch 404/500
18819/18819 [==============================] - 1s 62us/step - loss: 0.9956 - acc: 0.5121
Epoch 405/500
18819/18819 [==============================] - 1s 63us/step - loss: 0.9977 - acc: 0.5121
Epoch 406/500
18819/18819 [==============================] - 1s 62us/step - loss: 0.998

18819/18819 [==============================] - 1s 59us/step - loss: 0.9970 - acc: 0.5118
Epoch 477/500
18819/18819 [==============================] - 1s 61us/step - loss: 0.9958 - acc: 0.5121
Epoch 478/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9964 - acc: 0.5120
Epoch 479/500
18819/18819 [==============================] - 1s 58us/step - loss: 0.9965 - acc: 0.5121
Epoch 480/500
18819/18819 [==============================] - 1s 58us/step - loss: 0.9966 - acc: 0.5118
Epoch 481/500
18819/18819 [==============================] - 1s 59us/step - loss: 0.9971 - acc: 0.5121
Epoch 482/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9973 - acc: 0.5121: 0s - loss: 1.0053 -
Epoch 483/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9958 - acc: 0.5121
Epoch 484/500
18819/18819 [==============================] - 1s 60us/step - loss: 0.9948 - acc: 0.5120
Epoch 485/500
18819/18819 [==============================] - 1s 59

In [13]:
model.evaluate(x_test_snn, y_test_snn)

6274/6274 [==============================] - 0s 49us/step


[0.9959419040038474, 0.503984698756774]

In [14]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp, K.learning_phase()], outputs ) 

In [15]:
layer_outs_train = functor([x_train_snn, 1.])
layer_outs_test = functor([x_test_snn, 1.])

In [16]:
x_train_flat_snn = np.asarray(layer_outs_train[4]).reshape(-1,10)
x_test_flat_snn = np.asarray(layer_outs_test[4]).reshape(-1,10)

tsne = TSNE()
train_tsne_embeds_snn = tsne.fit_transform(x_train_flat_snn[:1000])
scatter(train_tsne_embeds_snn, y_train_snn[:1000], "Samples from Training Data LINCS (Triplet) after")

eval_tsne_embeds_snn = tsne.fit_transform(x_test_flat_snn[:1000])
scatter(eval_tsne_embeds_snn, y_test_snn[:1000], "Samples from Validation Data LINCS (Triplet) after")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
